In [ ]:
import math
import numpy as np
import pandas as pd
import cvxpy as cvx
from cvxopt import matrix
from cvxopt.modeling import variable
from cvxopt.modeling import op

## Exercise 3: Matching for a Dating Platform

In [ ]:
OD = pd.read_excel('OD.xlsx')
OD.head()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Name,Gender,Sports,Theater,Religion,Outdoors,Eating Out
0,Laura,Female,0,2,-1,0,2
1,Jennifer,Female,0,2,-1,-1,1
2,Emma,Female,2,-1,2,0,2
3,Olivia,Female,0,2,0,1,1
4,Martha,Female,0,-1,-1,0,0


In [ ]:
# data preprocessing
female = OD[OD['Gender'] == 'Female'].reset_index()
female_score = female.iloc[:, 3:]
male = OD[OD['Gender'] == 'Male'].reset_index()
male_score = male.iloc[:, 3:]

female_score = np.array(female_score)
male_score = np.array(male_score)

1. Assume all of the individuals in the data set would like to be matched to people of the opposite gender. What matches would you choose to maximize the aggregate match score?

In [ ]:
male['Name']

0     James
1    Robert
2     Peter
3      Eric
4    Daniel
5      Adam
6      Carl
7     Ralph
Name: Name, dtype: object

In [ ]:
# definition of the variables
matches = cvx.Variable((male.shape[0], female.shape[0]), boolean = True)

# definition of the constraints
constraints = [
    cvx.sum(matches, axis=1) == 1,
    cvx.sum(matches, axis=0) == 1
]

# definition of the objective function
obj = cvx.Maximize(cvx.sum(cvx.multiply(matches, male_score @ female_score.T)))

# solving the problem
prob = cvx.Problem(obj, constraints)
prob.solve()

print(f'Total match score: {prob.value:.2f}')
for i, j in np.argwhere(matches.value):
  print('Match:', male['Name'].iloc[i], 'and', female['Name'].iloc[j])

Total match score: 48.00
Match: James and Mary
Match: Robert and Sophia
Match: Peter and Olivia
Match: Eric and Martha
Match: Daniel and Emma
Match: Adam and Jennifer
Match: Carl and Laura
Match: Ralph and Isabella


2. How would the results change if each person is presented with two possible matches?

In [ ]:
# definition of the variables
matches = cvx.Variable((male.shape[0], female.shape[0]), boolean = True)

# definition of the constraints
constraints = [
    cvx.sum(matches, axis=1) == 2,
    cvx.sum(matches, axis=0) == 2,
]

# definition of the objective function
obj = cvx.Maximize(cvx.sum(cvx.multiply(matches, male_score @ female_score.T)))

# solving the problem
prob = cvx.Problem(obj, constraints)
prob.solve()

print(f'Total match score: {prob.value:.2f}')
for i in range(8):
    top2_matches = np.argsort(matches.value[i])[::-1][:2]
    print(male['Name'].iloc[i], 'top matches:')
    for j in top2_matches:
        print(' ', female['Name'].iloc[j])

Total match score: 82.00
James top matches:
  Mary
  Emma
Robert top matches:
  Sophia
  Jennifer
Peter top matches:
  Mary
  Isabella
Eric top matches:
  Sophia
  Martha
Daniel top matches:
  Martha
  Emma
Adam top matches:
  Jennifer
  Laura
Carl top matches:
  Olivia
  Laura
Ralph top matches:
  Isabella
  Olivia
